# 3.1 머신러닝을 사용해서 결측치 처리
- 메모리 에러가 남
- 코드 수정 필요
- 파일 저장위치: `../data/결측치_output/`
- 
https://medium.com/analytics-vidhya/handle-missing-values-in-categorical-features-b7c5b073dda2

## 데이터 불러오기

In [1]:
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv("../data/customer.csv")
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [6]:
df.drop(['케어솔루션고객여부', '거주행정동명', 'label_y', 'label_x'], axis = 1, inplace = True)

In [7]:
# 날짜형으로 변환
df['매출일자(배송완료일자)'] = pd.to_datetime(df['매출일자(배송완료일자)'], format='%Y-%m-%d')
df['년도'] = df['매출일자(배송완료일자)'].dt.year
df['월'] = df['매출일자(배송완료일자)'].dt.month

In [8]:
# 보유멤버십포인트 결측치는 0으로 채우기
df.loc[df['보유멤버십포인트'].isnull(), '보유멤버십포인트'] = 0

In [9]:
df_ = df.isnull().sum().sort_values().to_frame()

In [10]:
# column_name chagne
df_ = df_.rename(columns = {0 : "nullcount"})
df_ = df_.rename(columns = {"index" : "column_name"})
df_ = df_.T
df_

,고객ID,제휴가입여부,프리미엄카드보유여부,제휴카드보유여부,보유멤버십포인트,포인트사용빈도,년도,건강관리가전품목수,Only&Best품목수,주거형태,...,최근구매채널,금액,멤버십고객여부,멤버십앱설치여부,멤버십가입경로,거주광역명,거주시군구명,연령대,아파트평형대,아파트시세
nullcount,0,0,0,0,0,0,0,0,0,0,...,4,338,6905,6905,6905,18804,18870,27995,128164,128164


In [11]:
# int 변수로 바꿀 수 있는 str 변수를 찾아서 type 변환

df['케어솔루션품목수'] = df['케어솔루션품목수'].apply(lambda x : int(x[0]))
df['케어십품목수'] = df['케어십품목수'].apply(lambda x : int(x[0]))
df['Only&Best품목수'] = df['Only&Best품목수'].apply(lambda x : int(x[0]))
df['건강관리가전품목수'] = df['건강관리가전품목수'].apply(lambda x : int(x[0]))

In [12]:
pd.set_option('display.max_columns', None)
df.head(1)

,고객ID,매출일자(배송완료일자),타겟제품명,금액,멤버십고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,년도,월
0,34410,2022-04-29,에어컨,300만원 이상 400만원 이하,0.0,0,0,0,15KK이상20KK미만,온라인,1,1,30대,아파트,1~2억,20평대,미사용,0,베스트샵,0,0,0,0.0,전북,전주시 덕진구,2022,4


## 학습 데이터 구성

* 학습 데이터 : null_count가 없는 컬럼들만 모두 학습 데이터에 포함시켜야 되며, 이후 순차적으로 컬럼들을 예측하고, 컬럼들이 하나씩 추가되어 누적되는 방식

In [13]:
# 학습데이터

train_columns = [] # 학습 데이터 구성시, 활용할 컬럼명을 저장해주는 리스트
NA_columns = [] # 학습 데이터에 포함되지 않고, 추후에 순차적으로 예측할 컬럼명을 저장해주는 리스트 

for column in df_.columns:
    if df_.loc["nullcount", column] == 0:
        train_columns.append(column)
    else:
        NA_columns.append(column)

print("학습 데이터에 포함시킬 컬럼 : ", train_columns)
print("추후 예측할 NA 컬럼 : ", NA_columns)

학습 데이터에 포함시킬 컬럼 :  ['고객ID', '제휴가입여부', '프리미엄카드보유여부', '제휴카드보유여부', '보유멤버십포인트', '포인트사용빈도', '년도', '건강관리가전품목수', 'Only&Best품목수', '주거형태', '최근3년구매금액', '매출일자(배송완료일자)', '타겟제품명', '케어솔루션품목수', '월', '케어십가입여부', '케어십품목수']
추후 예측할 NA 컬럼 :  ['최근구매채널', '금액', '멤버십고객여부', '멤버십앱설치여부', '멤버십가입경로', '거주광역명', '거주시군구명', '연령대', '아파트평형대', '아파트시세']


In [14]:
# binary 변수 -> int type
if '멤버십고객여부' in train_columns:
    df['멤버십고객여부'] = df['멤버십고객여부'].apply(lambda x : int(int(x)))
if '멤버십앱설치여부' in train_columns:    
    df['멤버십앱설치여부'] = df['멤버십앱설치여부'].apply(lambda x : int(int(x)))
if '케어십가입여부' in train_columns:
    df['케어십가입여부'] = df['케어십가입여부'].apply(lambda x : int(int(x)))
if '제휴카드보유여부' in train_columns:
    df['제휴카드보유여부'] = df['제휴카드보유여부'].apply(lambda x : int(int(x)))
if '프리미엄카드보유여부' in train_columns:
    df['프리미엄카드보유여부'] = df['프리미엄카드보유여부'].apply(lambda x : int(int(x)))
if '제휴가입여부' in train_columns:
    df['제휴가입여부'] = df['제휴가입여부'].apply(lambda x : int(int(x)))
                                          

In [15]:
# 멤버십가입경로 4개 그룹으로 나눔

def change_variable(x):
    if x in ['베스트샵', '하이마트', '백화점', '전자랜드', '홈플러스', '(주)이마트']:
        return '매장'
    elif x in ['신한카드', '제휴카드', '하나SK카드', '롯데카드', 'LG페이', '네이버페이', '페이코', '스마트월렛']:
        return '카드'
    elif x in ['디얼디어', '바른손', '모닝글로리']:
        return '기타'
    elif x in ['LGE회원통합', 'LGE앱', 'LGE홈페이지(모바일)', '모바일그룹', '얍_KT', '얍_LGU', 'LGE홈페이지']:
        return '홈페이지'
    
def change_variable2(x):
    if x in ['백화점']:
        return '1'
    elif x in ['할인점 기타', '이마트', '온라인']:
        return '2'
    elif x in ['홈플러스', '하이마트', '전자랜드']:
        return '3'
    elif x in ['B2B', '베스트샵', '인터넷/홈쇼핑']:
        return '4'

In [16]:
df['멤버십가입경로'] = df['멤버십가입경로'].apply(lambda x: change_variable(x))
df['최근구매채널'] = df['최근구매채널'].apply(lambda x: change_variable2(x))
df.head(1)

,고객ID,매출일자(배송완료일자),타겟제품명,금액,멤버십고객여부,케어솔루션품목수,케어십가입여부,케어십품목수,최근3년구매금액,최근구매채널,Only&Best품목수,건강관리가전품목수,연령대,주거형태,아파트시세,아파트평형대,포인트사용빈도,보유멤버십포인트,멤버십가입경로,제휴카드보유여부,프리미엄카드보유여부,제휴가입여부,멤버십앱설치여부,거주광역명,거주시군구명,년도,월
0,34410,2022-04-29,에어컨,300만원 이상 400만원 이하,0.0,0,0,0,15KK이상20KK미만,2,1,1,30대,아파트,1~2억,20평대,미사용,0,매장,0,0,0,0.0,전북,전주시 덕진구,2022,4


In [17]:
def train_val_test_split(df, train_columns, NA_columns):
    # 원핫 인코딩
    df_encoded = pd.get_dummies(df[train_columns])
    
    # train_columns 와 NA컬럼 중 하나를  (NA_coulmns[0]()) 합쳐준다.
    # NA_columns에서 순차적으로 가장 맨 앞에 컬럼부터 예측
    df_train_val = pd.concat([df_encoded, df[NA_columns[0]]], axis = 1)

    # NA가 있는 row 만 test로 빼준다.
    df_test = df_train_val[df_train_val.isnull().any(axis=1)]
    y_test = df_test[NA_columns[0]]
    X_test = df_test.drop(['고객ID', '매출일자(배송완료일자)', NA_columns[0]], axis =1)

    # NA가 없는 데이터프레임
    df_train_val = df_train_val.dropna()

    # NA가 없는 데이터프레임에서 train, val을 나눈다. 
    # 어떤 알고리즘을 쓸지 알기 위해
    y_train_val = df_train_val[NA_columns[0]]
    X_train_val = df_train_val.drop(['고객ID', '매출일자(배송완료일자)', NA_columns[0]], axis =1)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=42)
    
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [48]:
# 원핫 인코딩
df_encoded = pd.get_dummies(df[train_columns])
# df_encoded.head(3)

In [49]:
# train_columns 와 NA컬럼 중 하나를  (NA_coulmns[0]()) 합쳐준다.
# NA_columns에서 순차적으로 가장 맨 앞에 컬럼부터 예측
df_train_val = pd.concat([df_encoded, df[NA_columns[0]]], axis = 1)

In [50]:
# NA가 있는 row 만 test로 빼준다.
df_test = df_train_val[df_train_val.isnull().any(axis=1)]
y_test = df_test[NA_columns[0]]
X_test = df_test.drop(['고객ID', '매출일자(배송완료일자)', NA_columns[0]], axis =1)

In [51]:
# NA가 없는 데이터프레임
df_train_val = df_train_val.dropna()

# NA가 없는 데이터프레임에서 train, val을 나눈다. 
# 어떤 알고리즘을 쓸지 알기 위해
y_train_val = df_train_val[NA_columns[0]]
X_train_val = df_train_val.drop(['고객ID', '매출일자(배송완료일자)', NA_columns[0]], axis =1)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=42)

In [52]:
print("X_train 갯수 : ", len(X_train))
print("y_train 갯수 : ", len(y_train))
print("X_val 갯수 : ", len(X_val))
print("y_val 갯수 : ", len(y_val))
print("X_test 갯수 : ", len(X_test))
print("y_test 갯수 : ", len(y_test))

X_train 갯수 :  372834
y_train 갯수 :  372834
X_val 갯수 :  159787
y_val 갯수 :  159787
X_test 갯수 :  4
y_test 갯수 :  4


# 결측치 많은 순으로 학습 후 예측

In [19]:
# 분류 알고리즘
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier

# 분류용 평가 함수
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import joblib
import pickle

In [20]:
def train_fit(y_target, X_train, y_train):
    
    # 검증할 모델들 생성
    model1 = DecisionTreeClassifier(random_state=1)
    model2 = RandomForestClassifier(random_state=1)
    model3 = AdaBoostClassifier(random_state=1)
    model4 = LogisticRegression(random_state=1)
    model5 = GradientBoostingClassifier(random_state=1)

    model_list = [model1, model2, model3, model4, model5]


    model_save = []
    # 모델 이름 
    model_name = ['DT', 'RF', 'Ada', 'LR', 'GB']    
    # 학습용 데이터를 학습한다.
    for m1 in tqdm(model_list) :
        print(f'===Training data ... {m1}===')
        model = m1.fit(X_train, y_train)
        model_save.append(model)
    
    return model_save
        

    #'saved_model'은 저장할 파일의 이름이다.
#     with open(f'{y_target}_saved_model', 'wb') as f:
#         pickle.dump(m1, f)

In [21]:
def predict_val(train_fit, X_val):
    # 학습용 데이터를 통해 결과를 예측한다.
    pred_list = []

    for m1 in tqdm(train_fit) :
        pred = m1.predict(X_val)
        pred_list.append(pred)
    return pred_list

In [22]:
def evaluate(pred_list, y_target, y_val, average):

    # 평가
    # 평가 결과를 담을 딕셔너리
    dict1 = {
        'accuracy' : [],
        'precision' : [],
        'recall' : [],
        'f1' : [],
        # 'roc_auc' : [],
        'name' : []
    }
    model_name = ['DT', 'RF', 'Ada']
    model_name = ['KNN', 'LR', 'DT', 'RF']#, 'Ada']  
    # 각 모델별 예측 결과를 하나씩 추출하여 평가한다.
    for idx, p1 in tqdm(enumerate(pred_list)) :

        r1 = accuracy_score(y_val, p1)
        r2 = precision_score(y_val, p1, average=average)
        r3 = recall_score(y_val, p1, average=average)
        r4 = f1_score(y_val, p1, average=average)
    #     r5 = roc_auc_score(y_val, p1, average='macro')

        dict1['accuracy'].append(r1)
        dict1['precision'].append(r2)
        dict1['recall'].append(r3)
        dict1['f1'].append(r4)
        # dict1['roc_auc'].append(r5)
        dict1['name'].append(f'{y_target}-{model_name[idx]}')

    # 시각화
    df100 = pd.DataFrame(dict1)
    df100.to_csv(f'../data/{y_target}_예측결과.csv', encoding='utf-8-sig')
    df100.set_index('name', inplace=True)
    df100.sort_values('f1', ascending=False, inplace=True)
    return df100
#     df100.plot.bar()
#     plt.rcParams['font.family'] = 'AppleGothic'
#     plt.savefig(f'{y_target}_모델예측성능_그래프.png')
#     plt.show()
    

In [23]:
def predict_X_Test (eval_, fit_result, X_test):
    eval_.reset_index(inplace=True)
    highest_f1_name = eval_.loc[eval_['f1'].idxmax()]['name'].split('-')[1]

    if 'KNN' == highest_f1_name:
        p = fit_result[0].predict(X_test)
    elif 'LR' == highest_f1_name:
        p = fit_result[1].predict(X_test)
    elif 'DT' == highest_f1_name:
        p = fit_result[2].predict(X_test)
    elif 'RF' == highest_f1_name:
        p = fit_result[3].predict(X_test)
    elif 'Ada' == highest_f1_name:
        p = fit_result[4].predict(X_test)
    return p

In [24]:
NA_columns[0]

'최근구매채널'

## 결측치 적은 순으로 채우기

In [ ]:
while NA_columns:
    print(NA_columns[0])
    X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(df, train_columns, NA_columns)
    fit_ = train_fit(NA_columns[0], X_train, y_train)
    predict_ = predict_val(fit_, X_val.values)

    if NA_columns[0] in ['멤버십고객여부', '멤버십앱설치여부']:
        eval_ = evaluate(predict_, NA_columns[0], y_val, 'binary') # macro
    else:
        eval_= evaluate(predict_, NA_columns[0], y_val, 'macro')

    predict_y_ = predict_X_Test(eval_, fit_, X_test.values)
    
    index_ = pd.DataFrame(y_test.index)
    index_.to_csv(f'../data/{NA_columns[0]}_missing_index.csv', encoding='utf-8-sig')
    
    df.loc[y_test.index, NA_columns[0]] = predict_y_

    add = NA_columns.pop(0)
    train_columns.append(add)

In [ ]:
# df.to_csv('../data/final_df_m5_v2.csv', encoding='utf-8-sig')

## f1 기준으로 가장 높은 순으로 채우기

In [25]:
def train_val_test_split_one(df, train_columns, NA_columns):
    # 원핫 인코딩
    df_encoded = pd.get_dummies(df[train_columns])
    
    # train_columns 와 NA컬럼 중 하나를  (NA_coulmns[0]()) 합쳐준다.
    # NA_columns에서 순차적으로 가장 맨 앞에 컬럼부터 예측
    df_train_val = pd.concat([df_encoded, df[NA_columns]], axis = 1)

    # NA가 있는 row만 test로 빼준다.
    df_test = df_train_val[df_train_val.isnull().any(axis=1)]
    y_test = df_test[NA_columns]
    X_test = df_test.drop(['고객ID', '매출일자(배송완료일자)', NA_columns], axis =1)

    # NA가 없는 데이터프레임
    df_train_val = df_train_val.dropna()

    # NA가 없는 데이터프레임에서 train, val을 나눈다. 
    # 어떤 알고리즘을 쓸지 알기 위해
    y_train_val = df_train_val[NA_columns]
    X_train_val = df_train_val.drop(['고객ID', '매출일자(배송완료일자)', NA_columns], axis =1)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.3, random_state=42)
    
    return (X_train, X_val, X_test, y_train, y_val, y_test)

In [59]:
def evaluate_one(pred_list, y_target, y_val, average, order):

    # 평가
    # 평가 결과를 담을 딕셔너리
    dict1 = {
        'order' : [],
        'accuracy' : [],
        'precision' : [],
        'recall' : [],
        'f1' : [],
        # 'roc_auc' : [],
        'name' : []
    }
    model_name = ['DT', 'RF', 'Ada', 'LR', 'GB']    
    # 각 모델별 예측 결과를 하나씩 추출하여 평가한다.
    for idx, p1 in tqdm(enumerate(pred_list)) :

        r1 = accuracy_score(y_val, p1)
        r2 = precision_score(y_val, p1, average=average)
        r3 = recall_score(y_val, p1, average=average)
        r4 = f1_score(y_val, p1, average=average)
    #     r5 = roc_auc_score(y_val, p1, average='macro')

        dict1['order'].append(order)
        dict1['accuracy'].append(r1)
        dict1['precision'].append(r2)
        dict1['recall'].append(r3)
        dict1['f1'].append(r4)
        # dict1['roc_auc'].append(r5)
        dict1['name'].append(f'{y_target}-{model_name[idx]}')

    # 시각화
    df100 = pd.DataFrame(dict1)
    df100.to_csv(f'../data/{order}/예측결과_{order}_{y_target}.csv', encoding='utf-8-sig')
    df100.set_index('name', inplace=True)
    df100.sort_values('f1', ascending=False, inplace=True)
    return df100
#     df100.plot.bar()
#     plt.rcParams['font.family'] = 'AppleGothic'
#     plt.savefig(f'{y_target}_모델예측성능_그래프.png')
#     plt.show()
    

In [60]:
def predict_X_Test_one(df, order_, train_columns):
    
    dfp = pd.read_csv(f'../data/{order_}/{order_}_evaluation.csv', encoding='utf-8-sig')


    a = dfp.loc[dfp['f1'].idxmax()]['name']
    missing_fill_col = a.split('-')[0]
    missing_model = a.split('-')[1]

    X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split_one(df, train_columns, missing_fill_col)


    # 검증할 모델들 생성
    def predict_X_Test_one(df, order_, train_columns):
    
    dfp = pd.read_csv(f'../data/{order_}/{order_}_evaluation.csv', encoding='utf-8-sig')


    a = dfp.loc[dfp['f1'].idxmax()]['name']
    missing_fill_col = a.split('-')[0]
    missing_model = a.split('-')[1]

    X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split_one(df, train_columns, missing_fill_col)

    # 검증할 모델들 생성
    model1 = DecisionTreeClassifier(random_state=1)
    model2 = RandomForestClassifier(random_state=1)
    model3 = AdaBoostClassifier(random_state=1)
    model4 = LogisticRegression(random_state=1)
    model5 = GradientBoostingClassifier(random_state=1)

    model_list = [model1, model2, model3, model4, model5]



    if 'KNN' == missing_model:
        p = model_list[0].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'LR' == missing_model:
        p = model_list[1].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'DT' == missing_model:
        p = model_list[2].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'RF' == missing_model:
        p = model_list[3].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'Ada' == missing_model:
        p = model_list[4].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y
        
    return df, missing_fill_col


    if 'KNN' == missing_model:
        p = model_list[0].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'LR' == missing_model:
        p = model_list[1].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'DT' == missing_model:
        p = model_list[2].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'RF' == missing_model:
        p = model_list[3].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y

    elif 'Ada' == missing_model:
        p = model_list[4].fit(X_train, y_train)
        pre_y = p.predict(X_test)
        df.loc[y_test.index, missing_fill_col] = pre_y
        
    return df, missing_fill_col

## 첫번째 결측치 컬럼 추가 : 멤버십앱설치여부

In [ ]:
dict2 = {
        'order' : [],
        'accuracy' : [],
        'precision' : [],
        'recall' : [],
        'f1' : [],
        # 'roc_auc' : [],
        'name' : []
    }

In [ ]:
order = 'first'
for i in range(len(NA_columns)):
    fill_col = NA_columns[i]
    X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split_one(df, train_columns, fill_col)
    fit_ = train_fit(fill_col, X_train, y_train)
    predict_ = predict_val(fit_, X_val.values)

    if fill_col in ['멤버십고객여부', '멤버십앱설치여부']:
        eval_ = evaluate_one(predict_, fill_col, y_val, 'binary', order) # macro
    else:
        eval_= evaluate_one(predict_, fill_col, y_val, 'macro', order)

    eval_.reset_index(inplace=True)
    dict2['order'].append(eval_.loc[eval_['f1'].idxmax()]['order'])
    dict2['accuracy'].append(eval_.loc[eval_['f1'].idxmax()]['accuracy'])
    dict2['precision'].append(eval_.loc[eval_['f1'].idxmax()]['precision'])
    dict2['recall'].append(eval_.loc[eval_['f1'].idxmax()]['recall'])
    dict2['f1'].append(eval_.loc[eval_['f1'].idxmax()]['f1'])
    # dict1['roc_auc'].append(r5)
    dict2['name'].append(eval_.loc[eval_['f1'].idxmax()]['name'])

df200 = pd.DataFrame(dict2)
# df200.to_csv(f'../data/{order}/{order}_evaluation.csv', encoding='utf-8-sig')
    

In [ ]:
predict_X_Test_one(df, order, train_columns)

### train_columns, NA_columns 업데이트

In [ ]:
# train_columns.append(missing_fill_col)
# NA_columns.remove(missing_fill_col)

In [61]:
def auto_fill_missingcol(order, NA_columns):
    dict2 = {
            'order' : [],
            'accuracy' : [],
            'precision' : [],
            'recall' : [],
            'f1' : [],
            # 'roc_auc' : [],
            'name' : []
        }

    for i in range(len(NA_columns)):
        fill_col = NA_columns[i]
        print(f'--------{fill_col}--------')
        X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split_one(df, train_columns, fill_col)
        fit_ = train_fit(fill_col, X_train, y_train)
        predict_ = predict_val(fit_, X_val.values)

        if fill_col in ['멤버십고객여부', '멤버십앱설치여부']:
            eval_ = evaluate_one(predict_, fill_col, y_val, 'binary', order) # macro
        else:
            eval_= evaluate_one(predict_, fill_col, y_val, 'macro', order)

        eval_.reset_index(inplace=True)
        dict2['order'].append(eval_.loc[eval_['f1'].idxmax()]['order'])
        dict2['accuracy'].append(eval_.loc[eval_['f1'].idxmax()]['accuracy'])
        dict2['precision'].append(eval_.loc[eval_['f1'].idxmax()]['precision'])
        dict2['recall'].append(eval_.loc[eval_['f1'].idxmax()]['recall'])
        dict2['f1'].append(eval_.loc[eval_['f1'].idxmax()]['f1'])
        # dict1['roc_auc'].append(r5)
        dict2['name'].append(eval_.loc[eval_['f1'].idxmax()]['name'])

    df200 = pd.DataFrame(dict2)
    df200.to_csv(f'../data/{order}/{order}_evaluation.csv', encoding='utf-8-sig')


In [62]:
order = ['1st_', '2nd_', '3rd_', '4th_', '5th_', '6th_', '7th_', '8th_', '9th_', '10th_']

In [ ]:
for orde in tqdm(order):
    print(f'=========={orde}==========')
    auto_fill_missingcol(orde, NA_columns)
    df, mfc = predict_X_Test_one(df, orde, train_columns)
    train_columns.append(mfc)
    NA_columns.remove(mfc)
    print('=============')
    print(df.isnull().sum())
    print(f'train_columns: {train_columns}')
    print(f'NA_columns: {NA_columns}')
    print(f'채운 컬럼 : {mfc}')

  0%|          | 0/10 [00:00<?, ?it/s]

==========1st_==========
--------최근구매채널--------


  0%|          | 0/4 [00:00<?, ?it/s]

===Training data ... KNeighborsClassifier()===
===Training data ... LogisticRegression(random_state=1)===
===Training data ... DecisionTreeClassifier(random_state=1)===
===Training data ... RandomForestClassifier(random_state=1)===


  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

0it [00:00, ?it/s]

--------거주광역명--------


  0%|          | 0/4 [00:00<?, ?it/s]

===Training data ... KNeighborsClassifier()===
===Training data ... LogisticRegression(random_state=1)===
===Training data ... DecisionTreeClassifier(random_state=1)===
===Training data ... RandomForestClassifier(random_state=1)===


In [ ]:
df, mfc = predict_X_Test_one(df, orde, train_columns)

In [ ]:
# 파일 저장 (머신러닝으로 학습하고 예측한 결측치)
# df.to_csv(../data/결측치_output/결측치완료_df_m5_v2.csv)

In [ ]:
train_columns.append(mfc)
NA_columns.remove(mfc)

print(df.isnull().sum())
print(f'train_columns: {train_columns}')
print(f'NA_columns: {NA_columns}')
print(f'채운 컬럼 : {mfc}')